In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import os
import xml.etree.ElementTree as ET

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU...")
else:
    print("CUDA is not available. Training on CPU...")

In [ ]:
# Find the overlap
import os

# Function to get filenames from a folder
def get_filenames(folder_path):
    return set(os.listdir(folder_path))

# Replace these paths with your actual folder paths
folder1 = f'C:/Users/username/OneDrive/Desktop/BGprediction/FA_split'
folder2 = f'C:/Users/username/OneDrive/Desktop/BGprediction/ML_split'
folder3 = f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split'
folder4 = f'C:/Users/username/OneDrive/Desktop/BGprediction/FACM_split'

# Get filenames from each folder
filenames1 = get_filenames(folder1)
filenames2 = get_filenames(folder2)
filenames3 = get_filenames(folder3)
filenames4 = get_filenames(folder4)

# Find common filenames across all folders
common_filenames = filenames1 & filenames2 & filenames3 & filenames4


In [ ]:
random_filenames = ['1100.csv',
 '870.csv',
 '958.csv',
 '255.csv',
 '933.csv',
 '1718.csv',
 '367.csv',
 '854.csv',
 '95.csv',
 '1287.csv']

Align first, then split, otherwise the order can be strange

# CGM

In [ ]:
# # Define the directory path
# train_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\LB_split'  # Use a raw string for paths on Windows

# # List files without their extensions
# train_file_names = [os.path.splitext(file)[0] for file in os.listdir(train_directory_path)
#               if os.path.isfile(os.path.join(train_directory_path, file))]

# # Print the list of file names
# print(train_file_names)

subject = pd.read_csv(f"../LB_split/1100.csv")

In [ ]:
def preprocess_t1dexi(path):

    subject = pd.read_csv(path)
    subject['LBDTC'] = pd.to_datetime(subject['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
    # print(subject['date'][0])
    subject.sort_values('LBDTC', inplace=True)  # Sort the DataFrame by the 'date' column

    # # Assuming self.interval_timedelta is set, for example:
    # interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

    # # Create a list to store the results
    # res = []

    # # Initialize the first group
    # if not subject.empty:
    #     current_group = [subject.iloc[0]['LBORRES']]
    #     last_time = subject.iloc[0]['LBDTC']

    # # Iterate over rows in DataFrame starting from the second row
    # for index, row in subject.iloc[1:].iterrows():
    #     current_time = row['LBDTC']
    #     if (current_time - last_time) <= interval_timedelta:
    #         # If the time difference is within the limit, add to the current group
    #         current_group.append(row['LBORRES'])
    #     else:
    #         # Otherwise, start a new group
    #         res.append(current_group)
    #         current_group = [row['LBORRES']]
    #     last_time = current_time

    # # Add the last group if it's not empty
    # if current_group:
    #     res.append(current_group)
    
    # # Filter out groups with fewer than 10 glucose readings
    # res = [group for group in res if len(group) >= 10]

    return subject

In [ ]:
cgm = preprocess_t1dexi(f"../LB_split/1100.csv")

In [ ]:
cgm

# Meal

## Meal quality check

In [ ]:
import os
import pandas as pd

# Path to the directory containing your CSV files
directory_path = '../ML_split'

# List to store filenames with no "NA" in the "MLDOSE" column
valid_files = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the "MLDOSE" column has no "NA" values
        if df["MLDOSE"].notna().all():
            valid_files.append(filename)


In [ ]:
len(valid_files)

In [ ]:
def preprocess_t1dexi(path):
    subject = pd.read_csv(path)
    subject['LBDTC'] = pd.to_datetime(subject['LBDTC'], errors='coerce')  # Convert 'date' column to datetime if not already
    # print(subject['date'][0])
    subject.sort_values('LBDTC', inplace=True)  

In [ ]:
subject_meal = pd.read_csv(f"../ML_split/870.csv")
subject_meal

In [ ]:
subject_meal["MLDOSE"]